In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from joblib import load, dump

from src.scripts.artemis_data import get_ns_cols, get_email_cols

In [2]:
df = pd.read_csv("../data/processed/final_whois_data.csv")

In [3]:
label = 'community'
y = df[label]
X = df.drop(columns=label)

In [4]:
enc_dict = load("../models/enc_dict.joblib")
country_encoder = load('../models/country_encoder.joblib')

In [5]:
X

,domain,redacted,name_servers_1,name_servers_2,name_servers_3,name_servers_4,registrant_contact_name,registrar,updated_date,country,...,num_different_ns_domains,main_name_server_domain,serverDeleteProhibited,clientDeleteProhibited,serverRenewProhibited,clientRenewProhibited,clientTransferProhibited,num_emails,email_domains,num_email_domains
0,griffithspartners.com.au,0.0,T.AU,R.AU,Q.AU,S.AU,1,afilias australia pty ltd,2022-03-17,zz,...,1,t,1,0,1,0,0,0,email_nan,1
1,21fss.com,0.0,NS33.DOMAINCONTROL.COM,NS34.DOMAINCONTROL.COM,NaN,NaN,0,"godaddy.com, llc",2021-01-19,us,...,1,domaincontrol,0,1,0,1,1,1,godaddy.com,1
2,conniedunndesigns.com,0.0,NS1.DREAMHOST.COM,NS2.DREAMHOST.COM,NS3.DREAMHOST.COM,NaN,0,dreamhost,2021-08-09,us,...,1,dreamhost,0,0,0,0,1,4,dreamhost.com;proxy.dreamhost.com,2
3,corporatespending.com,0.0,NS1.P23.DYNECT.NET,NS2.P23.DYNECT.NET,NS3.P23.DYNECT.NET,NS4.P23.DYNECT.NET,0,"godaddy.com, llc",2020-11-08,us,...,1,dynect,0,1,0,1,1,1,godaddy.com,1
4,eraliving.com,0.0,NS65.WORLDNIC.COM,NS66.WORLDNIC.COM,NaN,NaN,0,"network solutions, llc",2018-03-07,us,...,1,worldnic,0,0,0,0,1,3,eraliving.com;web.com,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5633,ewakyc72.top,1.0,a.dnspod.com,b.dnspod.com,A.DNSPOD.COM,B.DNSPOD.COM,0,"nicenic international group co., limited",2021-08-04,ru,...,1,dnspod,0,1,0,0,1,1,nicenic.net,1
5634,ewazqx71.top,1.0,a.dnspod.com,b.dnspod.com,A.DNSPOD.COM,B.DNSPOD.COM,0,"nicenic international group co., limited",2021-08-04,ru,...,1,dnspod,0,1,0,0,1,1,nicenic.net,1
5635,rebornx.duckdns.org,1.0,NS1.DUCKDNS.ORG,NS2.DUCKDNS.ORG,NS3.DUCKDNS.ORG,NS4.DUCKDNS.ORG,0,gandi sas,2021-11-26,fr,...,1,duckdns,0,0,0,0,1,2,support.gandi.net;contact.gandi.net,2
5636,mail.dipiluminacion.com,0.0,NS1.MYHOSTINGPACK.COM,NS2.MYHOSTINGPACK.COM,ns1.myhostingpack.com,ns2.myhostingpack.com,0,akky online solutions s.a. de c.v.,2021-07-26,mx,...,1,myhostingpack,0,0,0,0,1,3,akky.mx,1


In [7]:
cols_to_drop = ['domain', 'updated_date', 'expiration_date', 'creation_date', 'days_since_creation', 'malicious',
                'min_entropy', 'max_entropy', 'avg_entropy', 'domain_count'
                ]
cols_to_drop += get_ns_cols(7)
cols_to_drop += get_email_cols(3)
X = X.drop(columns=cols_to_drop)


In [8]:
for col in enc_dict.keys():
    X[col] = enc_dict[col].transform(X[col])
for col in ['country', 'dns_rec_a_cc', 'dns_rec_mx_cc']:
    X[col] = country_encoder.transform(X[col])

In [9]:
X = X.fillna(-1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
clf = KNeighborsClassifier(weights='distance').fit(X_train, y_train)

In [12]:
clf.score(X_test, y_test)

0.6985815602836879

In [13]:
dump(clf, '../models/community_predictor.joblib')

['../models/community_predictor.joblib']